In [4]:
import sys, os, io, json
from neo4j import GraphDatabase
from py2neo import Graph
from pathlib import Path
from pandas import DataFrame
import pandas as pd
import networkx as nx

graph = Graph("bolt://sarevok:7687")
driver = GraphDatabase.driver('bolt://sarevok:7687', auth=None)


In [5]:
# Add num_desc to all onto classes, return the max value

df = DataFrame(graph.run("""
MATCH (ancestor:`ols:Class`)<-[:`biolink:broad_match`*1..]-(subclass:`ols:Class`)
WITH ancestor, count(DISTINCT subclass) AS num_desc
SET ancestor.num_desc = num_desc
RETURN max(num_desc) AS max_num_desc
""").data())

max_num_desc=df['max_num_desc'][0]
print(max_num_desc)



3440826


In [6]:
# For all onto classes set ic to a normalised value based on the number of descendants where 0 descendants = 1 ic
# upper level classes like owl:Thing, bfo continuant will have very low ic scores (< 0.02)

df = DataFrame(graph.run("""
MATCH (cl:`ols:Class`)
SET cl.ic = 1.0 - (cl.num_desc/$max_num_desc)
""", {
    'max_num_desc': float(max_num_desc)
}).data())



In [7]:

df = DataFrame(graph.run("""
CREATE INDEX ic FOR (n:GraphNode) ON (n.ic)
""").data())

# Which nodes have an IC score of less than 0.5?

In [8]:

df = DataFrame(graph.run("""
MATCH (n:GraphNode) WHERE n.ic < 0.5
RETURN n.`grebi:name`[0] AS name, n.ic AS ic
""").data())

print(df.to_markdown())

|    | name                          |          ic |
|---:|:------------------------------|------------:|
|  0 | bfo:continuant                | 0.0177263   |
|  1 | bfo:independent_continuant    | 0.0746248   |
|  2 | object                        | 0.0883079   |
|  3 | biological entity             | 0.0849909   |
|  4 | NCBI_taxonomy:131567          | 0.324593    |
|  5 | entity                        | 0           |
|  6 | entity                        | 2.64471e-05 |
|  7 | organismal entity             | 0.195561    |
|  8 | experimental factor           | 0.00919489  |
|  9 | Archaea or Eukaryota          | 0.488634    |
| 10 | Eukaryota                     | 0.492926    |
| 11 | material entity               | 0.0746315   |
| 12 | cellular organisms or viruses | 0.251545    |
| 13 | obo:ncbitaxon.owl             | 0.245625    |
| 14 | obi:organism                  | 0.245622    |
| 15 | Thing                         | 2.55753e-05 |


In [10]:

df = DataFrame(graph.run("""
MATCH (n:GraphNode) WHERE n.ic < 0.8
RETURN n.`grebi:name`[0] AS name, n.ic AS ic
""").data())

print(df.to_markdown())

|    | name                                               |          ic |
|---:|:---------------------------------------------------|------------:|
|  0 | entity                                             | 0           |
|  1 | Thing                                              | 2.55753e-05 |
|  2 | entity                                             | 2.64471e-05 |
|  3 | experimental factor                                | 0.00919489  |
|  4 | bfo:continuant                                     | 0.0177263   |
|  5 | bfo:independent_continuant                         | 0.0746248   |
|  6 | material entity                                    | 0.0746315   |
|  7 | biological entity                                  | 0.0849909   |
|  8 | object                                             | 0.0883079   |
|  9 | organismal entity                                  | 0.195561    |
| 10 | obi:organism                                       | 0.245622    |
| 11 | obo:ncbitaxon.owl              

In [18]:

df = DataFrame(graph.run("""
MATCH (n:GraphNode)-[r]-() WHERE n.ic < 0.8
CALL {
WITH r
DELETE r
} IN TRANSACTIONS OF 1 ROWS
WITH DISTINCT n
CALL {
WITH n
DELETE n
} IN TRANSACTIONS OF 1 ROWS
""").data())

print(df.to_markdown())


TransientError: [General.MemoryPoolOutOfMemoryError] The allocation of an extra 2.0 MiB would use more than the limit 13.7 GiB. Currently using 13.7 GiB. dbms.memory.transaction.total.max threshold reached